# Travail pratique n°2 d'apprentissage automatique 2 sur la regression logistique à une seule classe

Réalisé par MULAPI TITA Ketsia

# Importation des librairies

In [ ]:
import numpy as np
import sklearn.datasets as data
import matplotlib.pyplot as plt

# 1) Problème jouet

### Générez une base de données synthétiques

In [ ]:
'''
Les n nombres de 300 points que contiennent les classes

'''
n = 300
sigma = 0.7
a = np.array([-1,-1])
b = np.array([1,1])

X1 = sigma * np.random.randn(n,2) + a
X2 = sigma * np.random.randn(n,2) + b

X = np.concatenate((X1,X2),axis=0)
y = np.concatenate((np.zeros((n)),np.ones((n))), axis = 0)

### Affichez le nuage de points pour obtenir un résultat similaire à celui de la figure

In [ ]:
plt.scatter(X1[:,0],X1[:,1],c = 'r', s= 7)
plt.scatter(X2[:,0],X2[:,1],c = 'blue', s= 7)
plt.show()

## Avant d’implémenter votre propre estimation du modèle de regression logistique, vous allez tester celui de scikit-learn :

#### Importez la modèle de régression logistique de scikit-learn 1 et entrainez le sur la base synthé-tique que vous venez de créer

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(random_state=0).fit(X, y)

In [ ]:
model

In [ ]:
model.score(X,y)

In [ ]:
X

##### prédiction basée sur les classses

In [ ]:
y_pred = model.predict(X)

In [ ]:
len(y)

In [ ]:
print(len(y_pred[y_pred!=y]) ," est le nombre de mauvaise prédiction.")

##### prédiction basée sur les prédictions qui sont des probabilités d'appartenance à une classe.

In [ ]:
y_pred_proba = model.predict_proba(X)

In [ ]:
y_pred_proba

In [ ]:
yy_pred_prob = np.zeros(y.shape).astype("int")
for i in range(y_pred_proba.shape[0]):
    if(y_pred_proba[i,0]>y_pred_proba[i,1]):
        yy_pred_prob[i] = 0
    else :
        yy_pred_prob[i] = 1
        
print(len(yy_pred_prob[yy_pred_prob!=y]))

In [ ]:
err = len(yy_pred_prob[yy_pred_prob!=y])

In [ ]:
print("Conclusion : Avec Sklearn, dans les 2 cas, on a le même résultat : "+str(err)+" mauvaise prédictions.")

## Affichez la fonction de décision du modèle, comme montré en figure 2. Pour cela, il faut :

#### 1. Générer une grille de points 2D à l’aide de la fonction mesh que nous donnons ci-dessous.

In [ ]:
def mesh(X, h = 0.02):
    x_min , x_max = X[:, 0]. min() - .5, X[:, 0]. max() + .5
    y_min , y_max = X[:, 1]. min() - .5, X[:, 1]. max() + .5
    xx , yy = np.meshgrid(np.arange(x_min , x_max , h), np.arange(y_min , y_max , h))
    return np.c_[xx.ravel(), yy.ravel()], xx , yy

#### Rassembler les prédictions du modèle pour chacun des points de cette grille

### Afficher le nuage de points de la base d’apprentissage

#### Afficher les frontières de décisions en utilisant la fonction contourf de Matplotlib.

In [ ]:
from matplotlib.colors import ListedColormap

In [ ]:
def afficher_for_sklearn(X,y,model):
    
    zz, xx, yy = mesh(X)
    #
    Z = model.predict(zz)
    ZZ = model.predict_proba(zz)  
    #
    Z = Z.reshape((-1,1))
    Z = Z.reshape(xx.shape)
    #
    ZZ = ZZ[:,0].reshape(xx.shape)
    #
    
    # L'arrière plan
    colors1 = ('lightcoral', 'steelblue')
    cmap_me1 = ListedColormap(colors1[:len(np.unique(y))])
    # La frontière
    colors2 = ('white','white')
    cmap_me_f1 = ListedColormap(colors2[:len(np.unique(y))])
    # Les points
    colors3 = ('firebrick','royalblue')
    cmap_me_back3 = ListedColormap(colors3[:len(np.unique(y))])
    
        
    # sklearn
    cm = plt.cm.RdBu
    
    plt.figure(0)
    out = plt.contourf(xx, yy, Z, cmap=cm,alpha=0.65)
    plt.contour(out, cmap=cmap_me_f1)
    plt.scatter(X[:,0], X[:,1], c=y, cmap=cmap_me_back3, edgecolors="k")
    plt.title(" la fonction de décision si les prédictions sont des classes from sklearn")
    plt.show()   
    
    # L'arrière plan
    colors4 = ('lightcoral','steelblue')
    cmap_me4 = ListedColormap(colors4[:len(np.unique(y))])
    # La frontière
    colors5 = ('whitesmoke','white')
    cmap_me_f2 = ListedColormap(colors5[:len(np.unique(y))])      
    # Les points
    colors6 = ('royalblue','firebrick')
    cmap_me_back6 = ListedColormap(colors6[:len(np.unique(y))])  
    
    plt.figure(1)
    out = plt.contourf(xx, yy, ZZ, cmap=cm,alpha=0.65)
    #plt.contour(out,cmap=cmap_me_f2)
    plt.scatter(X[:,0], X[:,1], c=y, cmap=cmap_me_back6, edgecolors="k")
    plt.title(" la fonction de décision si les prédictions sont des probabilités d’appartenance à la classe from sklearn")    
    plt.show()
    

In [ ]:
afficher_for_sklearn(X,y,model)

### FIGURE 2 – À gauche : la fonction de décision si les prédictions sont des classes. À droite : la fonction de décision si les prédictions sont des probabilités d’appartenance à la classe λ1

. Implémentez le calcul du coût en complétant la fonction ci-dessous ci-dessous :

## 2. Régression logistique sur le problème jouet

Implémentez le calcul du coût en complétant la fonction ci-dessous ci-dessous :

In [ ]:
def logreg_cost(X, y, w):
    z = X@w # z(n,1) = X(n,d)@w(d,1)
    # insérer votre code ici : La fonction que l’on veut minimiser
    cout = np.sum(np.log(1 + np.exp(-z))) + np.sum(y*z) # page 20
    return cout # cout(n,1) =  (n,1) + ((n,1)*(n,1))

. Implémentez l’estimation des paramètres du modèle de regression logistique en utilisant la
méthode de descente de gradient :

In [ ]:
def logreg_grad_desc(X, y):
    d = X.shape[1] 
    w = np.random.randn(d) 
    w = w.reshape((-1,1)) # w(d,1)
    nb_iter = 100 # to tune
    pas = 0.01 # to tune
    beta = 0.9 # for backtracking if needed - to tune
    for i in range(nb_iter): # 100
        z = X@w # z(n,1) = X(n,d) @ w(d,1)
        # insérer votre code ici : Gradient (page 21)
        grad = X.T @ (y - (np.exp(-z) / (1 + np.exp(-z)))) # n(d,1) = X(d,n)@(y(n,1)-(z(n,1)/z(n,1)))
        w = w - pas * grad # w (d,1) = w (d,1) - pas(1,1) * (d,1)
    return w # on retourne w (d,1)

Affichez la fonction de décision et proposez un protocole judicieux permettant de tester la performance de votre modèle

In [ ]:
W = logreg_grad_desc(X, y) # l'apprentissage w(d,1)
cout = logreg_cost(X, y, W) # le cout cout(n,1)

In [ ]:
print(cout)

In [ ]:
def afficher_for_me_from_scratch(X,y,w):
    '''
    Cette Fonction de décision pour la regression logistiques, 
    permettant de tester le modèle ici, 
    représenté par les paramètres w
    '''
    # On peut d'abord chercher à observer ce qu'on obteine 
    # avec les données d'entraînement afin de comparer
    # ce qu'on peut obtenir conformément à Sklearn.
    
    xtest_grid, xx, yy  = mesh(X)
    z = xtest_grid@w
    exps = np.exp(-z)
    den = (1+exps)
    Z = 1/den
    Z = Z.reshape(xx.shape)    
    
    # sklearn
    cm = plt.cm.RdBu
    
    # L'arrière plan
    colors1 = ('lightcoral', 'steelblue')
    cmap_me1 = ListedColormap(colors1[:len(np.unique(y))])
    # La frontière
    colors2 = ('white','white')
    cmap_me_f1 = ListedColormap(colors2[:len(np.unique(y))])
    # Les points
    colors3 = ('firebrick','royalblue')
    cmap_me_back3 = ListedColormap(colors3[:len(np.unique(y))])
    
    plt.figure(0)
    out = plt.contourf(xx, yy, Z, cmap=cm,alpha=0.9)
    plt.contour(out, cmap=cmap_me_f1)
    plt.scatter(X[:,0], X[:,1], c=y, cmap=cmap_me_back3, edgecolors="k")
    plt.title(" la fonction de décision si les prédictions sont des probas (from scratch)")
    
    plt.show() 
    
    # On va chercher à créer cette fois ci, 
    # des données qui serviront de test, mais pour se faire, il faudrait
    # que ces données réflètent notre réalité, c'est à dire
    # que ces données doivent correspondres aux données d'entrianement
    
    n = 100 # On crée alors 100 exemples à tester
    sigma = 0.7 # avec le même écart
    
    # Dans les mêmes intervals
    Xtest1 = sigma * np.random.randn(n,2) + a
    Xtest2 = sigma * np.random.randn(n,2) + b

    X_test = np.concatenate((Xtest1,Xtest2),axis=0)
    y_test = np.concatenate((np.zeros((n)),np.ones((n))), axis = 0)

    Xtest_Grid, xxx, yyy  = mesh(X_test)
    # Fonction de prédiction Sigmoïde : renvoie les probas
    ZZ = 1/(1+ np.exp(-Xtest_Grid@w))
    ZZ = ZZ.reshape(xxx.shape)
    
    # L'arrière plan
    colors1 = ('lightcoral', 'steelblue')
    cmap_me1 = ListedColormap(colors1[:len(np.unique(y))])
    # La frontière
    colors2 = ('white','white')
    cmap_me_f1 = ListedColormap(colors2[:len(np.unique(y))])
    # Les points
    colors3 = ('firebrick','royalblue')
    cmap_me_back3 = ListedColormap(colors3[:len(np.unique(y))])
    
    plt.figure(0)
    out = plt.contourf(xxx, yyy, ZZ, cmap=cm,alpha=0.9)
    plt.contour(out, cmap=cmap_me_f1)
    plt.scatter(X[:,0], X[:,1], c=y, cmap=cmap_me_back3, edgecolors="k")
    plt.title(" la fonction de décision si les prédictions sont des probas (from scratch)")
    
    plt.show()  

In [ ]:
afficher_for_me_from_scratch(X, y, W)

### 3. Diagnostique médical

On cherche à prédire la survenue d’un cancer à partir de paramètres médicaux. Pour cela, vous allez
utiliser la base de données breast_cancer de Scikit-learn.

3.1. Proposez une méthodologie valide pour comparer votre algorithme de régression logistique à
l’implémentation de scikit-learn. Les résultats seront présentés avec des taux de bonne classification moyen et des écart-types.

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#### Comparaison from SKlearn

In [ ]:
data = load_breast_cancer()

X = data.data
y = data.target

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(X).describe()

In [ ]:
pd.DataFrame(y).describe()

In [ ]:
plt.scatter(X[:,0],X[:,1])

On voit que les données ne ne sont pas centré à l'origine (normalisé) et n'ont pas la même échelle (standardiser) alors, on peut centrer et réduires les données :

3.1.1. Je normalise en suite mes données :

In [ ]:
X = (X - np.mean(X, axis=0))/np.std(X, axis=0)

In [ ]:
X

3.1.2. Je commence par séparer mes données en données de test et données dn'etraînement :

In [ ]:
def getTrainTest(X,y):
    '''
    renvoie X_train, X_test, y_train, y_test
    X_trian, y_train : Nos données d'apprentissage
    X_test, y_test : Nos données de test
    '''
    return train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train, X_test, y_train, y_test = getTrainTest(X, y)

In [ ]:
def cvComparaison(Model, X, y, kfolds):
    scoreCV = cross_val_score(Model,X,y,cv=kfolds)
    mean = np.mean(scoreCV)
    std = np.std(scoreCV)
    return mean, std

In [ ]:
def printResult(model, m,s):
    print(str(model)+" Accuracy: "+str(m)+" (+/- "+str(s)+")")

### 3.1.3 utilisation de sklearn

In [ ]:
model_log_reg = LogisticRegression().fit(X_train,y_train)

In [ ]:
model_log_reg.score(X_train,y_train)

On tolère donc un minimum d'erreur, ce qui implique que l'on ne va pas sur apprendre et qu'on va plus au moins bien classer.

In [ ]:
def afficher_sklearn(X_test,y_test,X_train,y_train, X, y, model):
    
    #
    zz, xx, yy = mesh(X_test)
    #
    Z = model.predict(zz)
    ZZ = model.predict_proba(zz)  
    #
    Z = Z.reshape((-1,1))
    Z = Z.reshape(xx.shape)
    #
    ZZ = ZZ[:,0].reshape(xx.shape)
    #
    
    # Mean, std
    
    zzz = cvComparaison(model, X_test, y_test, 5)
    ZZZZ = cvComparaison(model, X_test, y_test, 5)
    
    
    print("Ci-dessous, les focntions de décisions (classes, probas) avec leur mean (+/- std) respectif")
    
    # L'arrière plan
    colors1 = ('lightcoral', 'steelblue')
    cmap_me1 = ListedColormap(colors1[:len(np.unique(y))])
    # La frontière
    colors2 = ('white','white')
    cmap_me_f1 = ListedColormap(colors2[:len(np.unique(y))])
    # Les points
    colors3 = ('firebrick','royalblue')
    cmap_me_back3 = ListedColormap(colors3[:len(np.unique(y))])
    
        
    # sklearn
    cm = plt.cm.RdBu
    
    plt.figure(0)
    out = plt.contourf(xx, yy, Z, cmap=cm,alpha=0.65)
    plt.contour(out, cmap=cmap_me_f1)
    plt.scatter(X_test[:,0], X_test[:,1], c=y_test, cmap=cmap_me_back3, edgecolors="k")
    plt.title(" la fonction de décision si les prédictions sont des classes from sklearn")
    plt.show()       
    
    printResult(model, zzz[0],zzz[1])
    
    # L'arrière plan
    colors4 = ('lightcoral','steelblue')
    cmap_me4 = ListedColormap(colors4[:len(np.unique(y))])
    # La frontière
    colors5 = ('whitesmoke','white')
    cmap_me_f2 = ListedColormap(colors5[:len(np.unique(y))])      
    # Les points
    colors6 = ('royalblue','firebrick')
    cmap_me_back6 = ListedColormap(colors6[:len(np.unique(y))])  
    
    plt.figure(1)
    out = plt.contourf(xx, yy, ZZ, cmap=cm,alpha=0.65)
    #plt.contour(out,cmap=cmap_me_f2)
    plt.scatter(X_test[:,0], X_test[:,1], c=y_test, cmap=cmap_me_back6, edgecolors="k")
    plt.title(" la fonction de décision si les prédictions sont des probabilités d’appartenance à la classe from sklearn")    
    plt.show()
    
    
    printResult(model, zzz[0],zzz[1])

Ici, on test nos données X_test et y_test, à l'aide du modèle entraîné.

In [ ]:
afficher_sklearn(X_test,y_test,X_train,y_train, X, y, model_log_reg)

### Conclusion : 

On voit que le modèle de sklearn entraîné classe (décide) plutot bien les données, on a les mêmes moyennes et écarts types aussi bien avec le premier que le second. Et, le fait d'avoir tolérer un miniimum d'erreur, a pemris de ne pas sur apprendre et donc de bien décider.

#### Comparaison from algo by me

In [ ]:
def tx_Erreur(y, ypred):
    n = len(y)
    rejet = 0
    erreur = 0
    for i in range(n):
        if y_pred[i] == 2:
            rejet += 1
        elif y_pred[i] != y[i]:
            erreur += 1    
    return erreur/n, rejet/n

In [ ]:
def from_probas_to_class(X_test, y, w):
    z = np.zeros((X_test.shape[0],1))
    z = X_test@w
    y_pred = np.zeros(len(y))
    p1, p2 = np.zeros(len(y)), np.zeros(len(y))
    for i in range (len(X_test)):
        p1 = 1/(1 + np.exp(-z[i]))
        p2 = 1 - 1/(1+np.exp(z[i]))
        if(p1[i] >= 0.7):
            y_pred[i] = 0
        elif(p2[i] < 0.3):
            y_pred[i] = 1
        else:  # zone de confusion !!!
            y_pred[i] = 2
    return y_pred

In [ ]:
W = logreg_grad_desc(X_train, y_train)

In [ ]:
def another_log_reg_by_me(X_test, y_test, X_train, y_train):
    scores = []
    d = X_train.shape[1]
    W = logreg_grad_desc(X_train, y_train)
    y_pred = from_probas_to_class(X_test, y_test, W)
    rejet, erreur = tx_Erreur(y_test, y_pred)
    scores.append(1 - erreur)
    return np.mean(scores), np.std(scores)

In [ ]:
another_log_reg_by_me(X_test, y_test, X_train, y_train)

3.2. Comparez maintenant avec un algorithme la regression logistique régularisée en utilisant l’implémentation de Scikit-learn

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X_train, y_train)
y_pred_lasso = lasso_reg.predict(X_test)

In [ ]:
lasso_reg.score(X_train, y_train)

In [ ]:
print("le nombre d'erreur de prédiction est ", len(y_pred_lasso[y_pred_lasso!=y_test]))

# Conclusion

Notre modèle de regression logistique a sur appris et renvoie un score moyen de 1.

Celui de sklearn est optimisé, il n'est pas mal ne sur apprend pas et renvoie un score moyen en 0.96 et 0.97

Enfin la regression regularisé devrait être bonne mais lorsqu'elle n'est pas bien paramétré, elle n'est pas top.

# Difficulté rencontrées :

Tantôt le code fonctionne, tantôt il affiche une MemoryError
parfois il ne parvient pas à faire de reshape.
J'ai aussi rencontré un problème de np.exp overflow
je ne sais pas pourquoi et ne peux donc pas faire autrement(...).